In [1]:
import pandas as pd
import sys
import os
from pathlib import Path

project_root = str(Path.cwd().parent)
sys.path.append(project_root)
from sklearn.model_selection import train_test_split
from src.data.make_dataset import extract_departure_time
from src.data.make_dataset import impute_distance_by_average
from src.data.make_dataset import clean_cabin_type
from src.data.make_dataset import clean_data
from src.models.train_model import train_model
from src.models.predict_model import predict_model

In [2]:
airport_combined_path = '../data/processed/all_airports.csv'

In [3]:
airport_df = pd.read_csv(airport_combined_path)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
airport_df.sample(10)

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
2201498,cc27b762ff258212805a2be6322188d0,2022-04-28,2022-06-06,CLT,JFK,PT6H42M,False,False,False,480.09,548.0,1654510320||1654529700,2022-06-06T06:12:00.000-04:00||2022-06-06T11:3...,1654515300||1654534440,2022-06-06T07:35:00.000-04:00||2022-06-06T12:5...,IAD||JFK,CLT||IAD,United||Delta,UA||DL,Airbus A320||Canadair Regional Jet 900,4980||4740,327||221,coach||coach
2335217,b54cbf91b994775bbe3698468e837950,2022-05-02,2022-06-12,CLT,MIA,PT5H58M,False,False,False,220.10,663.0,1655063280||1655080320,2022-06-12T15:48:00.000-04:00||2022-06-12T20:3...,1655069040||1655084760,2022-06-12T17:24:00.000-04:00||2022-06-12T21:4...,MCO||MIA,CLT||MCO,American Airlines||American Airlines,AA||AA,Airbus A321||Boeing 737-800,5760||4440,464||199,coach||coach
8093067,60598ccfe11073c33a26bc512a06fc6c,2022-05-07,2022-05-08,LAX,DTW,PT4H23M,False,False,True,308.59,NaN,1652029140,2022-05-08T09:59:00.000-07:00,1652044920,2022-05-08T17:22:00.000-04:00,DTW,LAX,Spirit Airlines,NK,NaN,15780,NaN,coach
9887931,522d52f794bdb25ea3bae7d927bc4ba6,2022-04-30,2022-05-09,MIA,LAX,PT10H19M,False,False,False,612.60,2353.0,1652088000||1652112900,2022-05-09T05:20:00.000-04:00||2022-05-09T11:1...,1652099700||1652125140,2022-05-09T07:35:00.000-05:00||2022-05-09T12:3...,DFW||LAX,MIA||DFW,American Airlines||American Airlines,AA||AA,Boeing 737-800||Airbus A321,11700||12240,1115||1238,coach||coach
5303910,cf63c0ffb27612aa683f807451b92de8,2022-04-23,2022-05-31,EWR,DFW,PT3H54M,False,False,True,206.60,1380.0,1654019880,2022-05-31T13:58:00.000-04:00,1654033920,2022-05-31T16:52:00.000-05:00,DFW,EWR,American Airlines,AA,Boeing 737-800,14040,1380,coach
10822009,0cbec0f7bfd099f4e0d45d1112810b1c,2022-05-12,2022-05-28,OAK,LAX,PT9H31M,False,False,False,477.10,1432.0,1653761400||1653778620||1653789600,2022-05-28T11:10:00.000-07:00||2022-05-28T15:5...,1653767100||1653785040||1653795660,2022-05-28T12:45:00.000-07:00||2022-05-28T17:4...,PDX||SFO||LAX,OAK||PDX||SFO,Alaska Airlines||United||United,AS||UA||UA,Embraer 175||Boeing 737-900||Embraer 175 (Enha...,5700||6420||6060,543||550||339,coach||coach||coach
8316889,86de8933f5c54bd90aeadc89e4e28736,2022-05-12,2022-06-26,LAX,PHL,PT7H17M,False,False,False,714.60,2536.0,1656252000||1656266400,2022-06-26T07:00:00.000-07:00||2022-06-26T13:0...,1656263820||1656278220,2022-06-26T12:17:00.000-05:00||2022-06-26T17:1...,DFW||PHL,LAX||DFW,American Airlines||American Airlines,AA||AA,||Airbus A321,11820||11820,1238||1298,coach||coach
9416257,19ad1b65e127f9dad72e42ad08e5fe36,2022-05-13,2022-06-29,LGA,LAX,PT7H48M,False,False,False,452.60,2465.0,1656502200||1656515100,2022-06-29T07:30:00.000-04:00||2022-06-29T10:0...,1656511740||1656530280,2022-06-29T09:09:00.000-05:00||2022-06-29T12:1...,ORD||LAX,LGA||ORD,American Airlines||American Airlines,AA||AA,Boeing 737-800||Boeing 737-800,9540||15180,720||1745,coach||coach
10632457,60060a32de60754725468d4223cab881,2022-05-01,2022-06-01,OAK,ATL,PT13H49M,False,False,False,565.71,2175.0,1654088160||1654115340||1654127280,2022-06-01T05:56:00.000-07:00||2022-06-01T14:2...,1654095300||1654120500||1654137900,2022-06-01T08:55:00.000-06:00||2022-06-01T15:5...,SLC||DEN||ATL,OAK||SLC||DEN,Delta||United||United,DL||UA||UA,Embraer 175 (Enhanced Winglets)||Airbus A320||...,7140||5160||10620,588||380||1207,coach||coach||coach
5976317,962dffa5a4dc9fe208cf7a690a10db69,2022-04-23,2022-06-14,IAD,BOS,PT9H,True,False,False,284.20,1488.0,1655205000||1655228400,2022-06-14T07:10:00.000-04:00||2022-06-14T13:4...,1655211720||1655237400,2022-06-14T09:02:00.000-04:00||2022-06-14T16:1...,ATL||BOS,IAD||ATL,Delta|

In [6]:
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13519999 entries, 0 to 13519998
Data columns (total 23 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   legId                              object 
 1   searchDate                         object 
 2   flightDate                         object 
 3   startingAirport                    object 
 4   destinationAirport                 object 
 5   travelDuration                     object 
 6   isBasicEconomy                     bool   
 7   isRefundable                       bool   
 8   isNonStop                          bool   
 9   totalFare                          float64
 10  totalTravelDistance                float64
 11  segmentsDepartureTimeEpochSeconds  object 
 12  segmentsDepartureTimeRaw           object 
 13  segmentsArrivalTimeEpochSeconds    object 
 14  segmentsArrivalTimeRaw             object 
 15  segmentsArrivalAirportCode         object 
 16  segmentsDepartur

In [7]:
import pandas as pd

def calculate_average_distances(airport_df):
    """
    Calculates the average distance between unique origin and destination airports.

    Args:
        airport_df: A pandas DataFrame with 'Origin', 'Destination', and 'Distance' columns.

    Returns:
        A pandas DataFrame with unique origin and destination airports and their average distance.
    """
    average_distances = airport_df.groupby(['startingAirport', 'destinationAirport'])['totalTravelDistance'].mean().reset_index()

    return average_distances

In [8]:
average_distances_df = calculate_average_distances(airport_df)
print(average_distances_df)

    startingAirport destinationAirport  totalTravelDistance
0               ATL                BOS           998.032995
1               ATL                CLT           662.624730
2               ATL                DEN          1546.128489
3               ATL                DFW           907.045841
4               ATL                DTW           973.100301
5               ATL                EWR           828.117167
6               ATL                IAD           621.642341
7               ATL                JFK           962.933895
8               ATL                LAX          2086.698655
9               ATL                LGA           869.946287
10              ATL                MIA          1146.822334
11              ATL                OAK          2422.447664
12              ATL                ORD           963.911324
13              ATL                PHL           872.197084
14              ATL                SFO          2480.410744
15              BOS                ATL  

In [9]:
Airport_df_cleaned = clean_data(airport_df)

In [10]:
Airport_df_cleaned.head()

,startingAirport,destinationAirport,Departure_Year,Departure_Month,Departure_Day,depatureTimeCategory,Cabin_Type,Cabin_Score,totalFare,totalTravelDistance
0,ATL,BOS,2022,4,17,Afternoon,coach,0,248.6,947.0
1,ATL,BOS,2022,4,17,Morning,coach,0,248.6,947.0
2,ATL,BOS,2022,4,17,Afternoon,coach,0,248.6,947.0
3,ATL,BOS,2022,4,17,Evening,coach,0,248.6,947.0
4,ATL,BOS,2022,4,17,Afternoon,coach,0,248.6,947.0


In [11]:
Airport_df_cleaned.columns

Index(['startingAirport', 'destinationAirport', 'Departure_Year',
       'Departure_Month', 'Departure_Day', 'depatureTimeCategory',
       'Cabin_Type', 'Cabin_Score', 'totalFare', 'totalTravelDistance'],
      dtype='object')

In [12]:
X = Airport_df_cleaned.drop(['totalFare'], axis=1)  # remove target variable
y = Airport_df_cleaned['totalFare']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2,  # 20% for testing
    random_state=42  # for reproducibility
)

In [14]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [15]:
model = lgb.LGBMRegressor(
    objective='regression',
    random_state=42,
    n_jobs=-1
)

In [16]:
pipeline = train_model(model, X_train, y_train, output_path='../models/')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.095004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 45
[LightGBM] [Info] Number of data points in the train set: 10815999, number of used features: 8
[LightGBM] [Info] Start training from score 373.752004


In [17]:
results = predict_model(X_test, y_test, pipeline)


Model Performance:
RMSE: $201.06
MAE: $148.90
R2 Score: 0.062
MAPE: 57.97%


In [19]:
from src.models.train_model import tune_pipeline

In [20]:
param_grid = {
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__n_estimators': [100, 500, 1000],
    'model__num_leaves': [31, 62, 127],
    'model__max_depth': [-1, 5, 10],
    'model__min_child_samples': [20, 50, 100],
    'model__subsample': [0.6, 0.8, 1.0],
    'model__colsample_bytree': [0.6, 0.8, 1.0]
}

# Tune the pipeline
results = tune_pipeline(
    pipeline=pipeline,
    param_grid=param_grid,
    X_train=X_train,
    y_train=y_train,
    n_iter= 5,
    search_type='random',  # or 'random'
    cv=5
)

# Get the best pipeline
best_pipeline = results['best_pipeline']


Starting random search CV...
Fitting 5 folds for each of 5 candidates, totalling 25 fits


c:\Users\harry\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030598 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 41
[LightGBM] [Info] Number of data points in the train set: 10815999, number of used features: 6
[LightGBM] [Info] Start training from score 373.752004
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

OSError: Cannot save file into a non-existent directory: 'models'

In [ ]:
results_tune = predict_model(X_test, y_test, best_pipeline)